# Extract Frames from Videos and save it in a folder with the emotion label
This file also saves the MainDataSaved.csv file which contains the path to the frames and the emotion label

In [1]:
import pandas as pd
from collections import Counter
import numpy as np
import ast
import matplotlib.pyplot as plt
import csv
import os
import cv2

## Read Annotations File

In [6]:

videos_folder = 'Videos' # change the folder name to the folder where the videos are stored
outputs_path ='ExtractedData'

typeEmotions=['Anger','Disgust','Fear','Happy','Sad','Surprise','Neutral']

annotations = pd.read_csv("./Annotations/annotations.csv", header=None)
annotations.columns = ["video_tag", "clip_id", "label", "frame_no", "x", "y", "w", "h", "person_id"]

In [7]:
uniqueVideos= annotations['video_tag'].unique()

In [8]:
#remove 'No annotation' from the label column, the column is a list of values and selecting the most frequent annotation out of the 3 annotations
annotations['label'] = annotations['label'].apply(ast.literal_eval)

#remove 'No annotation' from the label column, the column is a list of values
annotations['label'] = annotations['label'].apply(lambda x: [i for i in x if i != 'No annotation'])

annotations['label'] = annotations['label'].apply(lambda x: [item for sublist in x for item in sublist])
def most_common(lst):
    counts = Counter(lst)
    return max(counts, key=counts.get)
annotations['Unique_labels'] = [most_common(x) for x in annotations['label']]

In [9]:
annotations

,video_tag,clip_id,label,frame_no,x,y,w,h,person_id,Unique_labels
0,aJKL0ahn1Dk,1,"[Happy, Happy, Happy]",19532,41.965200,4.873195,44.216991,94.802684,0,Happy
1,aJKL0ahn1Dk,1,"[Happy, Happy, Happy]",19538,41.564836,4.874640,44.216991,94.802684,0,Happy
2,aJKL0ahn1Dk,1,"[Happy, Happy, Happy]",19544,41.164472,4.876086,44.216991,94.802684,0,Happy
3,aJKL0ahn1Dk,1,"[Happy, Happy, Happy]",19550,40.764108,4.877532,44.216991,94.802684,0,Happy
4,aJKL0ahn1Dk,1,"[Happy, Happy, Happy]",19556,39.646728,5.014136,44.216991,94.802684,0,Happy
...,...,...,...,...,...,...,...,...,...,...
8082,ngITkMvWuq8,3,"[Happy, Neutral]",22471,0.052083,6.782194,73.680363,93.125213,9,Happy
8083,ngITkMvWuq8,3,"[Happy, Neutral]",22477,0.052083,6.782194,72.978506,93.125213,9,Happy
8084,ngITkMvWuq8,3,"[Happy, Neutral]",22483,0.052083,6.782194,72.276649,93.125213,9,Happy
8085,ngITkMvWuq8,3,"[Happy, Neutral]",22489,3.314622,6.782194,69.153019,93.125213,9,Happy


# Save the frames instead of creating video

Saves the frames in a folder names as the video name, clip_id, person_id, and frame_no inside frames folder

In [13]:
import os
import cv2
import csv

# Assuming annotations, uniqueVideos, typeEmotions, and videos_folder are defined elsewhere
# Save the pathnames and emotions in a CSV file
PATHS = []
EMOTIONS = []

for video in uniqueVideos:
    print("------------------------------Processing video:-------------------------------", video)
    filterVideoData = annotations[annotations['video_tag'] == video]
    uniqueClipID = filterVideoData['clip_id'].unique()
    
    for clipID in uniqueClipID:
        
        print("Processing clip:", clipID)
        filterClipData = filterVideoData[filterVideoData['clip_id'] == clipID]
        uniquePersonID = filterClipData['person_id'].unique()

        for personID in uniquePersonID:
            print("Processing person:", personID)
            filterPersonData = filterClipData[filterClipData['person_id'] == personID]

            for emotion in typeEmotions:
                filterEmotionData = filterPersonData[filterPersonData['Unique_labels'] == emotion]
                print("Processing emotion:", emotion, ", len:", len(filterEmotionData))

                if len(filterEmotionData) > 0:
                    # Split frames into chunks of 18 or less
                    chunks = [filterEmotionData[i:i + 18] for i in range(0, len(filterEmotionData), 18)]

                    for frames in chunks:
                        # Create directory for frames
                        dir = os.getcwd().replace('/Extra', '')
                        pathname = f"{dir}/Frames/{video}_{frames.iloc[0]['clip_id']}_{personID}_{frames.iloc[0]['frame_no']}"
                        PATHS.append(pathname)
                        EMOTIONS.append(emotion)

                        # Check if path already exists
                        if os.path.exists(pathname):
                            print(f"Path {pathname} already exists, skipping.")
                            continue

                        # Create the directory if it doesn't exist
                        os.makedirs(pathname, exist_ok=True)
                        PATHS.append(pathname)
                        EMOTIONS.append(emotion)

                        # Load the video
                        video_path = f'./{videos_folder}/{video}/complete.mp4'
                        video_capture = cv2.VideoCapture(video_path)

                        if not video_capture.isOpened():
                            print(f"Failed to open video: {video_path}")
                            continue

                        # Average width and height for bounding box, if needed
                        maxWidthValue = frames['w'].mean()
                        maxHeightValue = frames['h'].mean()

                        # Process frames
                        for frame_num, x, y, w, h in zip(frames['frame_no'], frames['x'], frames['y'], frames['w'], frames['h']):
                            video_capture.set(cv2.CAP_PROP_POS_FRAMES, float(frame_num))
                            ret, frame = video_capture.read()

                            if not ret:
                                print(f"Frame {frame_num} could not be read from video.")
                                break

                            # Scale bounding box coordinates based on video frame dimensions
                            height, width = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
                            x, y = int((float(x) / 100) * width), int((float(y) / 100) * height)
                            w, h = int((float(w) / 100) * width), int((float(h) / 100) * height)

                            # Draw rectangle and crop frame
                            cv2.rectangle(frame, (x, y), (x + w, y + h), (128, 128, 128), 2)
                            cropped_frame = frame[y:y + h, x:x + w]

                            # Save the cropped frame
                            frame_path = f"{pathname}/{frame_num}.jpg"
                            cv2.imwrite(frame_path, cropped_frame)

                        video_capture.release()

# Save the paths and emotions to a CSV file
with open('frame_paths.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Path', 'Emotion'])

    for path, emotion in zip(PATHS, EMOTIONS):
        writer.writerow([path, emotion])
print("Processing completed.")


------------------------------Processing video:------------------------------- aJKL0ahn1Dk
Processing clip: 1
Processing person: 0
Processing emotion: Anger , len: 0
Processing emotion: Disgust , len: 0
Processing emotion: Fear , len: 0
Processing emotion: Happy , len: 77
Path /Users/poojakishore/Documents/Codes/CapitaSelecta/Frames/aJKL0ahn1Dk_1_0_19532 already exists, skipping.
Path /Users/poojakishore/Documents/Codes/CapitaSelecta/Frames/aJKL0ahn1Dk_1_0_19640 already exists, skipping.
Path /Users/poojakishore/Documents/Codes/CapitaSelecta/Frames/aJKL0ahn1Dk_1_0_19892 already exists, skipping.
Path /Users/poojakishore/Documents/Codes/CapitaSelecta/Frames/aJKL0ahn1Dk_1_0_20054 already exists, skipping.
Path /Users/poojakishore/Documents/Codes/CapitaSelecta/Frames/aJKL0ahn1Dk_1_0_20252 already exists, skipping.
Processing emotion: Sad , len: 0
Processing emotion: Surprise , len: 0
Processing emotion: Neutral , len: 0
Processing person: 1
Processing emotion: Anger , len: 0
Processing em

In [14]:
df = pd.DataFrame({'file_name':PATHS , 'label': EMOTIONS})
df.to_csv('./MainDataSaved.csv', index=False)

In [16]:
len(PATHS)

591